<a href="https://colab.research.google.com/github/achmfirmansyah/sweet_project/blob/master/ICST2020/01_Preparing_thesatelitte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
!earthengine authenticate

Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=1CZFZTMJ-814gbPR1TzX8jgsrUduxcsG45N8c6LyAjo&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1QEq3N-xayDcKmys7-JNa-K0DyZnkl7qzv7OyQcp7dtJfEhY9LB_O3o

Successfully saved authorization token.


In [ ]:
ee.Initialize()

In [ ]:
image1=ee.Image('users/achmfirmansyah/GHS_BUILT_LDSMT_GLOBE_R2018A_3857_30_V2_0_20_11')
image2=ee.Image('users/achmfirmansyah/GHS_BUILT_LDSMT_GLOBE_R2018A_3857_30_V2_0_21_11')
image3=ee.Image('users/achmfirmansyah/GHS_BUILT_LDSMT_GLOBE_R2018A_3857_30_V2_0_21_12')
image4=ee.Image('users/achmfirmansyah/GHS_BUILT_LDSMT_GLOBE_R2018A_3857_30_V2_0_22_11')
image5=ee.Image('users/achmfirmansyah/GHS_BUILT_LDSMT_GLOBE_R2018A_3857_30_V2_0_22_11')

GHSL_mosaic=ee.ImageCollection([image1,image2,image3,image4,image5]).mosaic().select('b1').rename('BU_class')

In [ ]:
jabodetabek=[[106.347,-5.906],[107.300,-6.823]]
gerbangkertasusila=[[112.355,-6.874],[113.163,-7.820]]
mebidangro=[[97.939,2.849],[98.932,3.897]]
sarbagita=[[114.926,-8.249],[115.379,-8.843]]
maminasata=[[119.196,-4.732],[120.034,-5.613]]
kedungsepur=[[109.962,-6.754],[111.264,-7.534]]
bandungraya=[[107.182,-6.691],[107.944,-7.314]]
data_name=['jabodetabek','gerbangkertasusila','mebidangro','sarbagita','maminasata','kedungsepur','bandungraya']
data_bound=[jabodetabek,gerbangkertasusila,mebidangro,sarbagita,maminasata,kedungsepur,bandungraya]
data_=[0,1,2,3,4,5,6]

In [ ]:
# LANDSAT 7 
landsat_7_2015=ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterDate('2019-01-01', '2019-12-31').select(['B4','B5','B6','B3']).reduce(ee.Reducer.median())
ndvi_landsat_7=landsat_7_2015.normalizedDifference(['B5_median','B4_median']).rename('NDVI_landsat_8')
ndbi_landsat_7=landsat_7_2015.normalizedDifference(['B6_median','B5_median']).rename('NDBI_landsat_8')
mndwi_landsat_7=landsat_7_2015.normalizedDifference(['B3_median','B6_median']).rename('MNDWI_landsat_8')
savi_landsat_7=landsat_7_2015.select('B5_median').subtract(landsat_7_2015.select('B4_median')).divide(landsat_7_2015.select('B5_median').add(landsat_7_2015.select('B4_median')).add(0.5)).multiply(1.5).rename('SAVI_landsat_8')
lse_landsat_7=landsat_7_2015.normalizedDifference(['B5_median','B4_median']).log().multiply(0.047).add(1.0094).rename('LSE_landsat_8')
landsat_7_oke=ee.Image.cat(ndvi_landsat_7,ndbi_landsat_7,mndwi_landsat_7,savi_landsat_7,lse_landsat_7).toDouble()

In [ ]:
#VIIRS
viirs_2015=ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate('2019'+'-01-01','2019'+'-12-31')
viirs_2015=viirs_2015.select('avg_rad')
median_viirs_2015=viirs_2015.reduce(ee.Reducer.median())

In [ ]:
project1=median_viirs_2015.projection();
GHSL_mosaic=GHSL_mosaic.reproject(crs=project1,scale=50).reduceResolution(reducer= ee.Reducer.mean(),maxPixels= 1024)
landsat_7_oke_mosaic=landsat_7_oke.reproject(crs=project1,scale=50).reduceResolution(reducer= ee.Reducer.mean(),maxPixels= 1024)
image_compiled=ee.Image.cat(GHSL_mosaic,landsat_7_oke_mosaic,median_viirs_2015).toDouble()
for i in data_:
  ee.batch.Export.image.toDrive(
      image=image_compiled,
      description='compiled_GHSL_30_train_2019_'+data_name[i],
      folder= 'Urban_',
      scale=30,
      crs='EPSG:4326',
      region=data_bound[i],
      fileFormat='GEO_TIFF').start()

In [ ]:
period_=[2014,2019]
for time_ in period_:
  print(time_)
  # LANDSAT 7 
  landsat_7_2015=ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterDate(str(time_)+'-01-01', str(time_)+'-12-31').select(['B4','B5','B6','B3']).reduce(ee.Reducer.median())
  ndvi_landsat_7=landsat_7_2015.normalizedDifference(['B5_median','B4_median']).rename('NDVI_landsat_8')
  ndbi_landsat_7=landsat_7_2015.normalizedDifference(['B6_median','B5_median']).rename('NDBI_landsat_8')
  mndwi_landsat_7=landsat_7_2015.normalizedDifference(['B3_median','B6_median']).rename('MNDWI_landsat_8')
  savi_landsat_7=landsat_7_2015.select('B5_median').subtract(landsat_7_2015.select('B4_median')).divide(landsat_7_2015.select('B5_median').add(landsat_7_2015.select('B4_median')).add(0.5)).multiply(1.5).rename('SAVI_landsat_8')
  lse_landsat_7=landsat_7_2015.normalizedDifference(['B5_median','B4_median']).log().multiply(0.047).add(1.0094).rename('LSE_landsat_8')
  landsat_7_oke=ee.Image.cat(ndvi_landsat_7,ndbi_landsat_7,mndwi_landsat_7,savi_landsat_7,lse_landsat_7).toDouble()
  #VIIRS
  viirs_2015=ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(str(time_)+'-01-01',str(time_)+'-12-31')
  viirs_2015=viirs_2015.select('avg_rad')
  median_viirs_2015=viirs_2015.reduce(ee.Reducer.median())
  project1=median_viirs_2015.projection();
  GHSL_mosaic=GHSL_mosaic.reproject(crs=project1,scale=50).reduceResolution(reducer= ee.Reducer.mean(),maxPixels= 1024)
  landsat_7_oke_mosaic=landsat_7_oke.reproject(crs=project1,scale=50).reduceResolution(reducer= ee.Reducer.mean(),maxPixels= 1024)
  image_compiled=ee.Image.cat(GHSL_mosaic,landsat_7_oke_mosaic,median_viirs_2015).toDouble()
  for i in data_:
    print(data_name[i])
    ee.batch.Export.image.toDrive(
        image=image_compiled,
        description='compiled_GHSL_30_train_'+str(time_)+'_'+data_name[i],
        folder= 'Urban_',
        scale=30,
        crs='EPSG:4326',
        region=data_bound[i],
        fileFormat='GEO_TIFF').start()

2013
jabodetabek
gerbangkertasusila
mebidangro
sarbagita
maminasata
kedungsepur
bandungraya
2019
jabodetabek
gerbangkertasusila
mebidangro
sarbagita
maminasata
kedungsepur
bandungraya


In [ ]:
# LANDSAT 7 
landsat_7_2015=ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterDate('2015-01-01', '2015-12-31').select(['B4','B5','B3','B2']).reduce(ee.Reducer.median())
ndvi_landsat_7=landsat_7_2015.normalizedDifference(['B4_median','B3_median']).rename('NDVI_landsat_7')
ndbi_landsat_7=landsat_7_2015.normalizedDifference(['B5_median','B4_median']).rename('NDBI_landsat_7')
mndwi_landsat_7=landsat_7_2015.normalizedDifference(['B2_median','B5_median']).rename('MNDWI_landsat_7')
savi_landsat_7=landsat_7_2015.select('B4_median').subtract(landsat_7_2015.select('B3_median')).divide(landsat_7_2015.select('B4_median').add(landsat_7_2015.select('B3_median')).add(0.5)).multiply(1.5).rename('SAVI_landsat_7')
lse_landsat_7=ndvi_landsat_7.select('NDVI_landsat_7').log().multiply(0.047).add(1.0094).rename('LSE_landsat_7')
landsat_7_oke=ee.Image.cat(ndvi_landsat_7,ndbi_landsat_7,mndwi_landsat_7,savi_landsat_7,lse_landsat_7).toDouble()
ee.batch.Export.image.toDrive(
    image=landsat_7_oke,
    description='landsat_7_calc',
    folder= 'Urban_',
    scale=1000,
    crs='EPSG:4326',
    region=[[106.314,-5.926],[107.300,-6.823]],
    fileFormat='GEO_TIFF').start()

In [ ]:
# VIIRS LST NDVI
## VIIRS
viirs_2015=ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate('2015'+'-01-01','2015'+'-12-31')
viirs_2015=viirs_2015.select('avg_rad')
median_viirs_2015=viirs_2015.reduce(ee.Reducer.median())
## LST
lst_2015=ee.ImageCollection("MODIS/006/MOD11A2").filterDate('2015'+'-01-01','2015'+'-12-31')
median_lst_2015=lst_2015.select('LST_Day_1km').reduce(ee.Reducer.median())
## NDVI
ndvi_2015=ee.ImageCollection("MODIS/006/MOD13A2").filterDate('2015'+'-01-01','2015'+'-12-31')
median_ndvi_2015=ndvi_2015.select('NDVI').reduce(ee.Reducer.median())
project1=median_ndvi_2015.projection();
median_viirs_2015 = viirs_2015.reduce(ee.Reducer.median()).reproject(crs=project1,scale=30).reduceResolution(reducer= ee.Reducer.mean(),maxPixels= 1112)
image_viirs_lst_ndvi=ee.Image.cat(median_ndvi_2015,median_lst_2015,median_viirs_2015).toDouble()
ee.batch.Export.image.toDrive(
    image=image_viirs_lst_ndvi,
    description='ndvi_lst_viirs_calc',
    folder= 'Urban_',
    scale=1000,
    crs='EPSG:4326',
    region=[[106.314,-5.926],[107.300,-6.823]],
    fileFormat='GEO_TIFF').start()

In [ ]:
#compiling_Landsat_MODIS_VIIRS
landsat_compiled=landsat_7_oke.reproject(crs=project1,scale=50).reduceResolution(reducer= ee.Reducer.mean(),maxPixels= 1024)
compiled=ee.Image.cat(landsat_compiled,image_viirs_lst_ndvi).toDouble()
ee.batch.Export.image.toDrive(
    image=compiled,
    description='compiled_calc_',
    folder= 'Urban_',
    scale=1000,
    crs='EPSG:4326',
    region=[[106.314,-5.926],[107.300,-6.823]],
    fileFormat='GEO_TIFF').start()

In [ ]:
#compiling_with_GHSL_combined
GHSL=ee.Image("users/achmfirmansyah/combined_GHSL").select('b1').rename('BU_density')
GHSL=GHSL.reproject(crs=project1,scale=50).reduceResolution(reducer= ee.Reducer.mean(),maxPixels= 1024)
compiled=ee.Image.cat(landsat_compiled,image_viirs_lst_ndvi,GHSL).toDouble()
ee.batch.Export.image.toDrive(
    image=compiled,
    description='ghsl_compiled_calc_',
    folder= 'Urban_',
    scale=1000,
    crs='EPSG:4326',
    region=[[106.314,-5.926],[107.300,-6.823]],
    fileFormat='GEO_TIFF').start()

In [ ]:
#compiling VIIRS with GHSL and Landsat Only
viirs_2015=ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate('2015'+'-01-01','2015'+'-12-31')
viirs_2015=viirs_2015.select('avg_rad')
median_viirs_2015=viirs_2015.reduce(ee.Reducer.median())
project_2=median_viirs_2015